- 스케줄 : 옵티마이져에서 적용
  - 학습률을 에포크별로 학습을 할때 동적으로 조정하는 기법
  - 지수감쇠 : ExponentialDecay
    - 학습률을 지수적으로 감소
  - 구간별 감쇠 : PiecewiseConstanctDecay
    - 구간별로 학습률을 다르게 적용
    - 특정스텝에서 학습률을 크게 변경하고자 할때 유용
  - 다항 감쇠 : PolynormialDecay
  - 코사인 감쇠 : CosinDecay
  - 사용자 지정(커스텀) : 함수를 구현해서 적용

In [5]:
# 예시....

import tensorflow as tf
import math
# 라이브러리
schedule_learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.1,
    staircase=True # 계단식 감소 , False는 연속 감소
)
tf.keras.optimizers.Adam(learning_rate = schedule_learning_rate)

#사용자 정의 함수
def exponential_decay_fn(epoch):
  return 0.01 * 0.1**(epoch/20)
ex_callback =  tf.keras.optimizers.schedules.LearningRateSchedule(exponential_decay_fn)
# model.fit(x_train,y_train, epochs=n_epocs, callbacks=[lr_callback])


# 사용자 정의 함수를 만들어서
# 구간별
def piecewise_constant_decay(epoch):
  if epoch < 5:
    return 0.01
  elif epoch < 15:
    return 0.005
  else:
    return 0.001

lr_callback =  tf.keras.optimizers.schedules.LearningRateSchedule(piecewise_constant_decay)
# model.fit(x_train,y_train, epochs=n_epocs, callbacks=[lr_callback])

- 신경망 학습 순서
  - 입력층 : 피처의 모양을 설정
    - 플렛, 정규화  
  - 은닉층 : 활성화 함수 Relu
    - 배치노말(사용할 경우 은닉층에서 활성화 함수를 사용 안하고)
    - 활성화층(activation)
  - 출력층  
    - 분류 (이진 - sigmoid, 다중 - softmax)
    - 회귀 ( 출력이 1)
           

  - 모델구성이 완료되면
  - 컴파일
    - 옵티마이져, 손실함수 , 평가방법
    - 스케줄
  - 모델 학습
    - 콜벡함수
      - 체크포인트
      - 조기종료
      - 사용자 정의 스케줄
  - 모델 평가
    - evaluate
  - 모델 적용
  - 모델 배포(도커, AWS, 기타등등.)



In [6]:
# CIFAR10
# 신경망
# 콜벡,배치노말,스케줄 등등 가능한 모든 자원을 사용해서 신경망 구성하고 학습
# 전이학습

In [22]:
fdata = tf.keras.datasets.fashion_mnist.load_data()
print(fdata[0][1].shape, fdata[0][1])

(60000,) [9 0 0 ... 3 0 5]


In [52]:
import tensorflow as tf
data = tf.keras.datasets.cifar10.load_data()
X = data[0][0]
y = data[0][1]

In [53]:
X.shape, y.shape

((50000, 32, 32, 3), (50000, 1))

In [54]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [35]:
# y_train = y_train.reshape(-1,)   # 손실함수 sparse_categorical_entropy
# y_test = y_test.reshape(-1,)

In [55]:
y_train = tf.keras.utils.to_categorical(y_train,10)  # categorical_entropy
y_test = tf.keras.utils.to_categorical(y_test,10)
y_train.shape, y_test.shape

((40000, 10), (10000, 10))

In [56]:
x_train.shape

(40000, 32, 32, 3)

In [37]:
input = tf.keras.layers.Input(shape=x_train.shape[1:])
flattern = tf.keras.layers.Flatten()
norm = tf.keras.layers.Normalization()
h1 = tf.keras.layers.Dense(100,activation='relu')
h2 = tf.keras.layers.Dense(100,activation='relu')
output = tf.keras.layers.Dense(10,activation='softmax')
# 조립
x = norm(input)
x = flattern(x)
x = h1(x)
x = h2(x)
output = output(x)
model = tf.keras.Model(input,output)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ normalization_4 (Normalization)      │ (None, 32, 32, 3)           │               7 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 3072)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 100)                 │         307,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 100)                 │          10,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 318,417 (1.21 MB)

 Trainable params: 318,410 (1.21 MB)

 Non-trainable params: 7 (32.00 B)

In [38]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10, batch_size = 64, validation_split=0.2)
model.evaluate(x_test,y_test)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1597 - loss: 64.4181 - val_accuracy: 0.1972 - val_loss: 2.5094
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.2071 - loss: 2.3941 - val_accuracy: 0.2256 - val_loss: 2.1679
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.2438 - loss: 2.1372 - val_accuracy: 0.2534 - val_loss: 2.1329
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.2603 - loss: 2.1039 - val_accuracy: 0.2481 - val_loss: 2.1720
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.2558 - loss: 2.1271 - val_accuracy: 0.2606 - val_loss: 2.0684
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.2620 - loss: 2.1752 - val_accuracy: 0.1018 - val_loss: 2.3210
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.0981 - loss: 2.3145 - val_accuracy: 0.1014 - val_loss: 2.3028
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.0960 - loss: 2.3026 - val_

[2.3032944202423096, 0.09730000048875809]

전이학습

In [57]:
from tensorflow.keras.applications import VGG16  # CNN

base_model = VGG16(include_top=False, input_shape=x_train.shape[1:])
base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)          │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# FC
x = base_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(100,activation='relu')(x)
output = tf.keras.layers.Dense(10,activation='softmax')(x)
model = tf.keras.Model(base_model.input, output)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)          │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 100)                 │          51,300 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 14,766,998 (56.33 MB)

 Trainable params: 14,766,998 (56.33 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
y_train.shape

(40000, 10)

In [ ]:
# 사전 학습된 가중치는 동결
for layer in base_model.layers:
  layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# 모델학습
hist = model.fit(x_train,y_train,epochs=10, batch_size = 64, validation_split=0.2)
# 모델 평가
model.evaluate(x_test,y_test)

Epoch 1/10
 24/500 ━━━━━━━━━━━━━━━━━━━━ 6:08 775ms/step - accuracy: 0.1607 - loss: 20.9216